## Importing data

In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Load the dataframe containing the comments and attribute labels (df can be found here: https://docs.google.com/spreadsheets/d/1eYL8xY4M245mN4IRAFfX1tDZ35zHx-or/edit?usp=sharing&ouid=110539186278461891801&rtpof=true&sd=true)
df = pd.read_excel("preprocessed_train.xlsx")
df = df.head(1000)
df["comment_text_lemm"] = df["comment_text_lemm"].astype(str)

# Splitting the dataset into training and testing sets
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)
display(train_df, test_df)

## Tokenizing and labeling of data

In [ ]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset
from transformers import BertTokenizer

# Loading the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

labels = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
labels = torch.tensor(labels, dtype=torch.float32)

# Splitting the data into train and test sets
train_labels, test_labels = train_test_split(labels, test_size=0.2, random_state=42)

# Tokenizing the comment text using the BERT tokenizer
train_encodings = tokenizer(train_df["comment_text_lemm"].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_df["comment_text_lemm"].tolist(), truncation=True, padding=True)

# Encoding the tokenized sequences using the BERT encoding scheme to obtain input features
train_input_ids = torch.tensor(train_encodings["input_ids"])
train_attention_mask = torch.tensor(train_encodings["attention_mask"])
test_input_ids = torch.tensor(test_encodings["input_ids"])
test_attention_mask = torch.tensor(test_encodings["attention_mask"])

# Making a TensorDataset for training and testing sets
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)


## Building BERT model and evaluation

In [ ]:
# Building a BERT-based NLP attention model with two layers ver2
class ToxicityClassifier(nn.Module):
    def __init__(self, num_labels):
        super(ToxicityClassifier, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

num_labels = 6  # Number of toxic attributes

In [ ]:
import torch
import torch.nn as nn
from transformers import BertForSequenceClassification, BertTokenizer

# DataLoader for training and testing sets
batch_size = 16  # Adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Making an instance of the ToxicityClassifier model
model = ToxicityClassifier(num_labels)

# Defining the optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()

# Training the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(5):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f"Epoch {epoch+1}/{5}: Average Loss = {avg_loss:.4f}")


In [ ]:
# Evaluating the model
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, _ = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        logits = model(input_ids, attention_mask)
        logits = torch.sigmoid(logits)
        predictions.extend(logits.cpu().numpy())

predictions = torch.tensor(predictions)


In [ ]:
# Assuming you have ground truth labels in `labels` (torch.Tensor or numpy.ndarray)

# Convert predictions to binary values (0 or 1) based on a threshold
threshold = 0.5
binary_predictions = (predictions >= threshold).to(torch.float32)

# Convert labels to numpy.ndarray if they are torch.Tensor
if isinstance(labels, torch.Tensor):
    labels = labels.cpu().numpy()

# Ensure that predictions and labels have the same number of samples
if len(binary_predictions) != len(labels):
    raise ValueError("Number of samples in predictions and labels does not match.")

# Calculate evaluation metrics
accuracy = accuracy_score(labels, binary_predictions)
precision = precision_score(labels, binary_predictions)
recall = recall_score(labels, binary_predictions)
f1 = f1_score(labels, binary_predictions)
roc_auc = roc_auc_score(labels, predictions)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")


## Finding toxic words for network

In [ ]:
import spacy
import re
from nltk.corpus import stopwords
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

train = df
train['comment_text'] = train['comment_text'].astype(str)
train

In [ ]:
train.index = train['id']
x_train = train['comment_text']
y_train = train.iloc[:, 2:]

In [ ]:
# if some kind of toxicity is detected, the sum across rows will yield one, 
# and the subtraction will give zero, and one otherwise
y_train['clean'] = 1 - y_train.sum(axis=1) >= 1  

In [ ]:
# the sum operation yield a series, and a series behaves like a dictionary
# as it has the items function that returns index-value tuples.
kinds, counts = zip(*y_train.sum(axis=0).items())

In [ ]:
bars = go.Bar(
        y=counts,
        x=kinds,
    )

layout = go.Layout(
    title="Class distribution in train set"
)

fig = go.Figure(data=[bars], layout=layout)
iplot(fig, filename='bar')

In [ ]:
import spacy
from nltk.corpus import stopwords

nlp = spacy.load("en_core_web_sm", disable=['parser', 'tagger', 'ner'])
stops = stopwords.words("english")

In [ ]:
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [ ]:
x_train_lemmatized = x_train.apply(normalize, lowercase=True, remove_stopwords=True)

In [ ]:
x_train_lemmatized.sample(1).iloc[0]

In [ ]:
from collections import Counter
word_counts = dict()

for kind in y_train.columns:
    word_counts[kind] = Counter()
    comments = x_train_lemmatized[y_train[kind]==1]
    for _, comment in comments.iteritems():
        word_counts[kind].update(comment.split(" "))
def most_common_words(kind, num_words=15):
    words, counts = zip(*word_counts[kind].most_common(num_words)[::-1])
    bars = go.Bar(
        y=words,
        x=counts,
        orientation="h"
    )

    layout = go.Layout(
        title="Most common words of the class \"{}\"".format(kind),
        yaxis=dict(
            ticklen=8  # to add some space between yaxis labels and the plot
        )
    )

    fig = go.Figure(data=[bars], layout=layout)
    iplot(fig, filename='bar')
most_common_words("toxic")

In [ ]:
most_common_words("severe_toxic")

In [ ]:
most_common_words("threat")

In [ ]:
most_common_words("clean")

In [ ]:
import pandas as pd

# Convert the 'comment_text' column to string
df['comment_text_lemm'] = df['comment_text_lemm'].astype(str)

# Concatenate all the toxic comments into a single string
all_toxic_comments = ' '.join(df[df['toxic'] == 1]['comment_text_lemm'])

# Split the string into individual words
all_toxic_words = all_toxic_comments.split()

# Calculate the frequency of each word
word_frequency = pd.Series(all_toxic_words).value_counts()

## Network visualization

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertTokenizer

# Get the word embeddings for toxic words
toxic_words = all_toxic_words

# Load the BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Tokenize the toxic words
tokenized_words = tokenizer(toxic_words, padding=True, truncation=True, return_tensors='pt')

# Forward pass to obtain word embeddings
with torch.no_grad():
    outputs = model(**tokenized_words)
    word_embeddings = outputs.last_hidden_state.squeeze(0)

# Reshape the word embeddings to remove the batch dimension
word_embeddings = word_embeddings.reshape(word_embeddings.size(0), -1)

# Calculate similarity scores between word embeddings
similarity_scores = cosine_similarity(word_embeddings.cpu().numpy())

# Create a graph
G = nx.Graph()

# Add nodes to the graph with word signatures
for i, word in enumerate(toxic_words):
    signature = tokenizer.convert_ids_to_tokens(tokenized_words['input_ids'][i].tolist())
    G.add_node(word, signature=signature)

# Add edges to the graph
for i, word1 in enumerate(toxic_words):
    for j, word2 in enumerate(toxic_words):
        if i != j:
            similarity = similarity_scores[i, j]
            G.add_edge(word1, word2, weight=similarity)

# Draw the graph
pos = nx.spring_layout(G, seed=42)
weights = [data['weight'] for _, _, data in G.edges(data=True)]
edges = nx.draw_networkx_edges(G, pos, edge_color=weights, edge_cmap=plt.cm.Reds, width=2)
nodes = nx.draw_networkx_nodes(G, pos, node_color='blue', alpha=0.7)

# Add word signatures to the node labels
node_labels = {node: ' '.join(G.nodes[node]['signature']) for node in G.nodes}
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=6)

# Add colorbar for edge weights
cbar = plt.colorbar(edges)
cbar.set_label('Connection Strength')

plt.title('Toxic Words Network')
plt.figure(figsize=(12, 8)) 
plt.axis('off')
plt.show()


In [ ]:
from bokeh.models import Circle, MultiLine, Range1d
from bokeh.plotting import figure, from_networkx, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import LabelSet

# Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

# Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

# Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

title = 'Toxic Words Network'

# Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
    ("Signature", "@signature"),
    #("Strongest connections to:", "@strongest_connections"),
]

# Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips=HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
              x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title=title)

# Create a network graph object
network_graph = from_networkx(G, nx.spring_layout, scale=2, center=(0, 0))

# Set plot background color
plot.background_fill_color = "white"

# Set node attributes
network_graph.node_renderer.glyph = Circle(size=15, fill_color="blue", fill_alpha=0.7)
network_graph.node_renderer.selection_glyph = Circle(size=15, fill_color="#17BEBB", fill_alpha=0.7)
network_graph.node_renderer.hover_glyph = Circle(size=15, fill_color="#456990", fill_alpha=0.7)

# Set edge attributes
network_graph.edge_renderer.glyph = MultiLine(line_color="#17BEBB", line_width=2)
network_graph.edge_renderer.selection_glyph = MultiLine(line_color="#17BEBB", line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color="#456990", line_width=2)

# Enable highlighting of connected nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

# Add the graph to the plot
plot.renderers.append(network_graph)

# Add word signatures as labels
node_labels = {
    node: ' '.join(
        word for word in G.nodes[node]['signature']
        if word not in ['[CLS]', '[SEP]', '[PAD]'] and not word.startswith('##')
    )
    for node in G.nodes
}

x, y = zip(*network_graph.node_renderer.data_source.data['index'])
source = ColumnDataSource({'x': x, 'y': y, 'word': list(node_labels.values()), 'signature': list(node_labels.values())})
labels = LabelSet(x='x', y='y', text='word', text_font_size='6pt', source=source, render_mode='canvas')


# Calculate the strongest connections for each node
strongest_connections = {}
for node in G.nodes:
    connections = G.edges(node, data=True)  # Include edge attributes
    strongest = sorted(connections, key=lambda x: x[2].get('weight', 0), reverse=True)[:5]  # Get top 5 connections by weight
    strongest_words = [target for _, target, attrs in strongest]
    strongest_connections[node] = strongest_words

# Add the strongest connections to the node data
source.data['strongest_connections'] = [strongest_connections.get(word, []) for word in source.data['word']]

# Show the plot
show(plot)


In [ ]:
from bokeh.models import Circle, MultiLine, Range1d
from bokeh.plotting import figure, from_networkx, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import LabelSet
from bokeh.palettes import Blues8
from bokeh.models.graphs import NodesAndLinkedEdges

# Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

# Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

title = 'Toxic Words Network'

# Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
    ("Signature", "@signature"),
    ("Strongest connections to:", "@strongest_connections"),
]

# Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips=HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
              x_range=Range1d(-1.5, 1.5), y_range=Range1d(-1.5, 1.5), title=title)

# Create a network graph object
network_graph = from_networkx(G, nx.spring_layout, scale=2, center=(0, 0))

# Set plot background color
plot.background_fill_color = "white"

# Calculate node sizes
node_sizes = [len(list(G.neighbors(node))) * 10 for node in G.nodes]
network_graph.node_renderer.glyph.size = {'field': size_by_this_attribute}

# Create a new list of node sizes for the selection and hover glyphs
selection_node_sizes = [size * 2 for size in node_sizes]
hover_node_sizes = [size * 1.5 for size in node_sizes]

# Set the selection and hover glyphs with the new sizes
network_graph.node_renderer.selection_glyph = Circle(size=selection_node_sizes, fill_color="#17BEBB", fill_alpha=0.7)
network_graph.node_renderer.hover_glyph = Circle(size=hover_node_sizes, fill_color="#456990", fill_alpha=0.7)

# Set edge attributes
network_graph.edge_renderer.glyph = MultiLine(line_color="#17BEBB", line_width=2)
network_graph.edge_renderer.selection_glyph = MultiLine(line_color="#17BEBB", line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color="#456990", line_width=2)

# Enable highlighting of connected nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

# Calculate the layout positions for the network graph
layout = nx.spring_layout(G, scale=2, center=(0, 0))
network_graph.layout_provider.graph_layout = layout

# Retrieve the x and y positions from the layout dictionary
x = [layout[node][0] for node in G.nodes]
y = [layout[node][1] for node in G.nodes]

# Update the ColumnDataSource with the x and y positions
network_graph.node_renderer.data_source.data['index_x'] = x
network_graph.node_renderer.data_source.data['index_y'] = y

# Add the graph to the plot
plot.renderers.append(network_graph)

# Calculate the strongest connections for each node
strongest_connections = {}
for node in G.nodes:
    connections = G.edges(node, data=True)  # Include edge attributes
    strongest = sorted(connections, key=lambda x: x[2].get('weight', 0), reverse=True)[:5]  # Get top 5 connections by weight
    strongest_words = [target for _, target, attrs in strongest]
    strongest_connections[node] = strongest_words

# Add the strongest connections to the node data
node_labels = {}
for node in G.nodes:
    signature = G.nodes[node].get('signature', '')
    words = [word for word in signature if word not in ['[CLS]', '[SEP]', '[PAD]'] and not word.startswith('##')]
    node_labels[node] = ' '.join(words)

source = ColumnDataSource({'x': x, 'y': y, 'word': list(node_labels.values()), 'signature': list(node_labels.values()),
                           'node_sizes': node_sizes, 'selection_node_sizes': selection_node_sizes,
                           'hover_node_sizes': hover_node_sizes})

labels = LabelSet(x='x', y='y', text='word', text_font_size='6pt', source=source, render_mode='canvas')

# Add the labels to the plot
plot.add_layout(labels)

# Show the plot
show(plot)
